In [1]:
import pandas as pd
import numpy as np  
import warnings
import re

warnings.filterwarnings('ignore')

raw_df = pd.read_excel('raw/푸드테크_H-PEACE_식생활관리_0923_최수연 - 전달.xlsx').drop(0, axis=0)
var_df = pd.read_excel('raw/변수 정의.xlsx')

info_var = ['R-ID', '수진일', '생년월', '성별', '나이', '신장', '최종학력', '결혼상태', '가계수입'] #
health_var = ['SBP', 'DBP', 'CHOL.', 'TG', 'LDL CHOL.', 'HDL CHOL.', 'GLUCOSE', 'HBA1C', 'eGFR', '허리둘레(WAIST)', '체중', '체질량지수']
disease_var = var_df[(var_df['분류'] == '질환')]['컬럼명1'].tolist()
medication_var = ['고혈압_투약여부','당뇨_투약여부', '고지혈증_투약여부']
smoke_var = var_df[(var_df['분류'] == '흡연')]['컬럼명1'].tolist()

diet_var = var_df[var_df['분류']=='식습관']
diet_var1 = diet_var['컬럼명1'].tolist()
diet_var2 = diet_var['컬럼명2'].tolist()

activity_var = var_df[var_df['분류']=='운동']['컬럼명1'].tolist()
drinking_var = var_df[(var_df['분류'] == '음주')]['컬럼명1'].tolist()

## 식습관 변수 통합

In [2]:
diet_df = raw_df[diet_var1 + diet_var2 + ['R-ID', '수진일']]

diet_var_g1 = diet_var.groupby('소분류')['컬럼명1'].apply(list).to_dict()
diet_var_g2 = diet_var.groupby('소분류')['컬럼명2'].apply(list).to_dict()

data_df1 = diet_df[diet_var1 + ['R-ID', '수진일']]
data_df2 = diet_df[diet_var2 + ['R-ID', '수진일']]

invalid_ids = set()

for subcat, cols in diet_var_g1.items():
    valid_cols = [c for c in cols if c in data_df1.columns]
    if not valid_cols:
        continue
    dup_mask = data_df1[valid_cols].notna().sum(axis=1) > 1
    dup_ids = set(data_df1.loc[dup_mask, 'R-ID'])
    invalid_ids.update(dup_ids)

for subcat, cols in diet_var_g2.items():
    valid_cols = [c for c in cols if c in data_df2.columns]
    if not valid_cols:
        continue
    dup_mask = data_df2[valid_cols].notna().sum(axis=1) > 1
    dup_ids = set(data_df2.loc[dup_mask, 'R-ID'])
    invalid_ids.update(dup_ids)

data_df1 = data_df1[~data_df1['R-ID'].isin(invalid_ids)]
data_df2 = data_df2[~data_df2['R-ID'].isin(invalid_ids)]

# score 맵핑
score_map1 = diet_var.set_index('컬럼명1')['score'].dropna()
score_map2 = diet_var.set_index('컬럼명2')['score'].dropna()

# diet_df1: 컬럼명1 기준
diet_df1 = pd.DataFrame(index=data_df1.index)
for subcategory, columns in diet_var_g1.items():
    valid_columns = [col for col in columns if col in data_df1.columns]
    diet_df1[subcategory] = (
        data_df1[valid_columns]
        .apply(lambda row: next(
            (score_map1.get(col) for col in valid_columns if pd.notna(row[col])), 
            np.nan
        ), axis=1)
    )

# diet_df2: 컬럼명2 기준
diet_df2 = pd.DataFrame(index=data_df2.index)
for subcategory, columns in diet_var_g2.items():
    valid_columns = [col for col in columns if col in data_df2.columns]
    diet_df2[subcategory] = (
        data_df2[valid_columns]
        .apply(lambda row: next(
            (score_map2.get(col) for col in valid_columns if pd.notna(row[col])), 
            np.nan
        ), axis=1)
    )

# 결합 및 최종 결과 생성
diet_df_cleaned = diet_df[~diet_df['R-ID'].isin(invalid_ids)]
diet_df_filled = pd.concat([diet_df2.fillna(diet_df1), diet_df_cleaned[['R-ID', '수진일']]],axis=1).dropna()

## 질환 변수 통합

In [3]:
info_df = raw_df[info_var]
disease_df_filled = raw_df[['R-ID', '수진일'] + health_var + disease_var + medication_var].dropna()
diet_disease_df_filled = disease_df_filled.merge(diet_df_filled, on=['R-ID', '수진일'], how='inner').merge(info_df, on=['R-ID', '수진일'], how='inner')

# 'nonHDLC' 추가
diet_disease_df_filled['CHOL.'] = pd.to_numeric(diet_disease_df_filled['CHOL.'], errors='coerce')
diet_disease_df_filled['HDL CHOL.'] = pd.to_numeric(diet_disease_df_filled['HDL CHOL.'], errors='coerce')
diet_disease_df_filled['nonHDLC'] = diet_disease_df_filled['CHOL.'] - diet_disease_df_filled['HDL CHOL.']

# 비만 추가
diet_disease_df_filled['비만'] = np.where(
    diet_disease_df_filled['체질량지수'] < 23, 0,
    np.where(diet_disease_df_filled['체질량지수'] >= 25, 2, 1)
)

# BMI category 추가
diet_disease_df_filled['BMI category'] = np.where(
    diet_disease_df_filled['체질량지수'] < 18.5, 0,
        np.where(diet_disease_df_filled['체질량지수'] < 23, 1,
            np.where(diet_disease_df_filled['체질량지수'] < 25, 2,
                np.where(diet_disease_df_filled['체질량지수'] < 30, 3, 4)
            )
        )
    )

# WC 추가
diet_disease_df_filled['허리둘레(WAIST)'] = pd.to_numeric(diet_disease_df_filled['허리둘레(WAIST)'], errors='coerce')
diet_disease_df_filled['WC (M>=90, F>=85)'] = np.where(
    ((diet_disease_df_filled['허리둘레(WAIST)'] >= 90) & (diet_disease_df_filled['성별'] == 'M')) |
    ((diet_disease_df_filled['허리둘레(WAIST)'] >= 85) & (diet_disease_df_filled['성별'] == 'F')), 1, 0
)

# 만성콩팥병 추가
diet_disease_df_filled['Chronic kidney disease (eGFR<60)'] = np.where(
    diet_disease_df_filled['eGFR'] < 60, 1,
    np.where(diet_disease_df_filled['eGFR'] >= 60, 0, np.nan)
).astype(int)

## 운동 변수 통합

In [4]:
activity_var1 = activity_var[:6]
activity_var2 = activity_var[9:]

activity_df1 = raw_df[activity_var1+ ['R-ID', '수진일']]
activity_df2 = raw_df[activity_var2+ ['R-ID', '수진일']]

activity_df_filled = activity_df2.fillna(activity_df1)

for col in activity_var2:
    activity_df_filled[col] = pd.to_numeric(activity_df_filled[col], errors="coerce")

activity_df_filled["중강도_총시간"] = activity_df_filled["중강도_운동빈도 [19ver only]"] * activity_df_filled["중강도_운동시간 [19ver only]"]
activity_df_filled["고강도_총시간"] = activity_df_filled["고강도_운동빈도 [19ver only]"] * activity_df_filled["고강도_운동시간 [19ver only]"]
activity_df_filled["총_운동시간"] = activity_df_filled["중강도_총시간"] + activity_df_filled["고강도_총시간"]

def categorize_activity(row):
    if row["중강도_운동여부 [19ver only]"] == 0 and row["고강도_운동여부 [19ver only]"] == 0:
        return 0
    elif (1 <= row["중강도_총시간"] < 150 or 1 <= row["고강도_총시간"] < 75 or 1 <= row["총_운동시간"] < 150):
        return 1
    elif (row["중강도_총시간"] >= 150 or row["고강도_총시간"] >= 75 or row["총_운동시간"] >= 150):
        return 2
    return np.nan

activity_df_filled["활동량"] = activity_df_filled.apply(categorize_activity, axis=1)
activity_df_filled = activity_df_filled[['활동량', 'R-ID', '수진일']].fillna('Missing value')

## 음주 변수 통합

In [5]:
drinking_df = raw_df[drinking_var+ ['R-ID', '수진일']]

# 음주 정의
ALCOHOL_PER_GLASS = 8  # 1잔 = 8g 알코올
FREQ_MAPPING = {
    0: 0,  # 월 1회 이하 (0회/주 로 계산)
    1: 0.75,  # 월 2-4회 (평균 0.75회/주로 계산)
    2: 2,     # 주 2회
    3: 3,     # 주 3회
    4: 4,     # 주 4회
    5: 5,     # 주 5회
    6: 6,     # 주 6회
    7: 7      # 주 7회
}
DRINK_MAPPING = {
    0: 2,    # 2잔
    1: 4,    # 4잔
    2: 6,    # 6잔
    3: 9,    # 9잔
    4: 10    # 10잔 이상
}

# 음주량 계산 및 카테고리 분류 함수
def classify_alcohol_intake(row):
    if pd.isna(row['음주빈도']) or pd.isna(row['음주량']):
        return np.nan
    
    weekly_frequency = FREQ_MAPPING.get(row['음주빈도'], np.nan)
    drinks_per_session = DRINK_MAPPING.get(row['음주량'], np.nan)
    
    if pd.isna(weekly_frequency) or pd.isna(drinks_per_session):
        return np.nan
    
    weekly_alcohol_intake = weekly_frequency * drinks_per_session * ALCOHOL_PER_GLASS
    
    if weekly_alcohol_intake == 0:
        return 0
    elif weekly_alcohol_intake < 210:
        return 1
    else:
        return 2

drinking_df['음주빈도'] = pd.to_numeric(drinking_df['음주빈도'], errors='coerce')
drinking_df['음주량'] = pd.to_numeric(drinking_df['음주량'], errors='coerce')

drinking_df['음주'] = drinking_df.apply(classify_alcohol_intake, axis=1)
drinking_df = drinking_df[['음주', 'R-ID', '수진일']].fillna('Missing value')

## 흡연 변수 통합

In [6]:
smoke_df = raw_df[smoke_var + ['R-ID', '수진일']]

diet_disease_smoke_df_filled = diet_disease_df_filled.merge(smoke_df, on=['R-ID', '수진일'], how='inner')

# 0=비흡연, 1=과거흡연, 2=현재흡연, na -> 0=현재 비흡연, 1=현재흡연, na=na
for col in smoke_var:
    diet_disease_smoke_df_filled[col] = diet_disease_smoke_df_filled[col].apply(
        lambda x: 1 if x == 2 else (0 if pd.notna(x) else 'Missing value')
    )

## 전체 통합

In [7]:
total_df = diet_disease_smoke_df_filled.merge(diet_df_filled, on=['R-ID', '수진일'], how='inner', suffixes=('', '_y')).filter(regex='^(?!.*_y$)')\
                  .merge(activity_df_filled, on=['R-ID', '수진일'], how='inner')\
                  .merge(drinking_df, on=['R-ID', '수진일'], how='inner')
total_df.sort_values(by='수진일', ascending=True, inplace=True)

counts = total_df.groupby('R-ID').size().reset_index(name='counts')
again = counts[counts['counts']>1]['R-ID'].tolist()
total_df_again = total_df[total_df['R-ID'].isin(again)]

In [8]:
len(again)

11238

In [17]:
T1_T0_data = []
total_df_again['수진일'] = pd.to_datetime(total_df_again['수진일'])

for patient_id, patient_data in total_df_again.groupby('R-ID'):
    patient_data = patient_data.sort_values('수진일')
    for i in range(1, len(patient_data)):
        prev_visit = patient_data.iloc[i-1]
        curr_visit = patient_data.iloc[i]
        days_between = (curr_visit['수진일'] - prev_visit['수진일']).days          
        row = {'patient_id': patient_id, 'days_between': days_between}        
        T1_T0_data.append(row)

pd.DataFrame(T1_T0_data)

,patient_id,days_between
0,R-495-00003951,411
1,R-495-00003951,1045
2,R-495-00003951,908
3,R-495-00005135,1470
4,R-495-00009421,365
...,...,...
17855,R-495-00183991,843
17856,R-495-00184050,1085
17857,R-495-00184056,1465
17858,R-495-00184200,265


In [18]:
total_df_again.to_excel('total_again_an.xlsx')
total_df_again.drop(['최종학력', '결혼상태', '가계수입'],axis=1).to_excel('total_again.xlsx')

In [9]:
'''snuh_col = ['R-ID', '수진일', '곡류', '단백질류', '채소', '과일', '유제품', '단맛', '튀김', '고지방 육류', '인스턴트 가공식품', '짠 식습관', '음료류', '짠 간']
diet_df_filled = diet_df_filled[snuh_col]

def transform_values(df):
    # Grain Products: 3→5, 2→3, 1→1
    df['Grain Products'] = df['Grain Products'].replace({3: 5, 2: 3, 1: 1})
    
    # Protein Foods: 4→5, 3→3, 2 or 1→1
    df['Protein Foods'] = df['Protein Foods'].replace({4: 5, 2: 1, 1: 1})
    
    # Vegetables: 4→5, 3→3, 2 or 1→1
    df['Vegetables'] = df['Vegetables'].replace({4: 5, 2: 1, 1: 1})
    
    # Fruits: 3→5, 2→3, 1→1
    df['Fruits'] = df['Fruits'].replace({3: 5, 2: 3})
    
    # Dairy Products: 3→5, 4 or 2→3, 1→1
    df['Dairy Products'] = df['Dairy Products'].replace({3: 5, 4: 3, 2: 3})
    
    # Sweet Food Consumption: 1→5, 2→3, 3→1
    df['Sweet Food Consumption'] = df['Sweet Food Consumption'].replace({1: 5, 2: 3, 3: 1})
    
    # Fried Foods: 1→5, 2→3, 3 or 4→1
    df['Fried Foods'] = df['Fried Foods'].replace({1: 5, 2: 3, 3: 1, 4: 1})
    
    # High Fat Meat: 1→5, 2→3, 3 or 4→1
    df['High Fat Meat'] = df['High Fat Meat'].replace({1: 5, 2: 3, 3: 1, 4: 1})
    
    # Processed Foods: 1→5, 2→3, 3 or 4→1
    df['Processed Foods'] = df['Processed Foods'].replace({1: 5, 2: 3, 3: 1, 4: 1})
    
    # Salty Food Consumption: 1→5, 2→3, 3→1
    df['Salty Food Consumption'] = df['Salty Food Consumption'].replace({1: 5, 2: 3, 3: 1})
    
    # Sugar-Sweetened Beverages: 1→5, 2→3, 3 or 4→1
    df['Sugar-Sweetened Beverages'] = df['Sugar-Sweetened Beverages'].replace({1: 5, 2: 3, 3: 1, 4: 1})
    
    # Additional Salt Use: 1→5, 2→3, 3→1
    df['Additional Salt Use'] = df['Additional Salt Use'].replace({1: 5, 2: 3, 3: 1})
    
    return df'''

"snuh_col = ['R-ID', '수진일', '곡류', '단백질류', '채소', '과일', '유제품', '단맛', '튀김', '고지방 육류', '인스턴트 가공식품', '짠 식습관', '음료류', '짠 간']\ndiet_df_filled = diet_df_filled[snuh_col]\n\ndef transform_values(df):\n    # Grain Products: 3→5, 2→3, 1→1\n    df['Grain Products'] = df['Grain Products'].replace({3: 5, 2: 3, 1: 1})\n    \n    # Protein Foods: 4→5, 3→3, 2 or 1→1\n    df['Protein Foods'] = df['Protein Foods'].replace({4: 5, 2: 1, 1: 1})\n    \n    # Vegetables: 4→5, 3→3, 2 or 1→1\n    df['Vegetables'] = df['Vegetables'].replace({4: 5, 2: 1, 1: 1})\n    \n    # Fruits: 3→5, 2→3, 1→1\n    df['Fruits'] = df['Fruits'].replace({3: 5, 2: 3})\n    \n    # Dairy Products: 3→5, 4 or 2→3, 1→1\n    df['Dairy Products'] = df['Dairy Products'].replace({3: 5, 4: 3, 2: 3})\n    \n    # Sweet Food Consumption: 1→5, 2→3, 3→1\n    df['Sweet Food Consumption'] = df['Sweet Food Consumption'].replace({1: 5, 2: 3, 3: 1})\n    \n    # Fried Foods: 1→5, 2→3, 3 or 4→1\n    df['Fried Foods'] = df['Fried Foods'].replace({1: 5